# Sprint8 アンサンブル学習

# ライブラリのimport

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.linear_model import PassiveAggressiveRegressor, ARDRegression, RidgeCV
from sklearn.linear_model import TheilSenRegressor, RANSACRegressor, HuberRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, ExtraTreesRegressor, HistGradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor, GradientBoostingRegressor, VotingRegressor, StackingRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# データの読み込み＆作成

In [2]:
# 読み込み
data = pd.read_csv('train.csv')
data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [3]:
# 前処理（説明省略　対数変換は場合によっては不要かもしれない）
mini_data = data[['GrLivArea','YearBuilt','SalePrice']]
log_min_data = mini_data.apply(np.log1p)
X = log_min_data[['GrLivArea','YearBuilt']].values
y = log_min_data['SalePrice'].values
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8,random_state=None)

3種類のアンサンブル学習をスクラッチ実装していきます。そして、それぞれの効果を小さめのデータセットで確認します。


ブレンディング
バギング
スタッキング

小さなデータセットの用意
以前も利用した回帰のデータセットを用意します。


House Prices: Advanced Regression Techniques


この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使います。


train.csvを学習データ8割、検証データ2割に分割してください。


scikit-learn
単一のモデルはスクラッチ実装ではなく、scikit-learnなどのライブラリの使用を推奨します。


sklearn.linear_model.LinearRegression — scikit-learn 0.21.3 documentation


sklearn.svm.SVR — scikit-learn 0.21.3 documentation


sklearn.tree.DecisionTreeRegressor — scikit-learn 0.21.3 documentation


3.ブレンディング

## 【問題1】ブレンディングのスクラッチ実装
ブレンディング をスクラッチ実装し、単一モデルより精度があがる例を 最低3つ 示してください。精度があがるとは、検証データに対する平均二乗誤差（MSE）が小さくなることを指します。


ブレンディングとは
ブレンディングとは、N個の多様なモデルを独立して学習させ、推定結果を重み付けした上で足し合わせる方法です。最も単純には平均をとります。多様なモデルとは、以下のような条件を変化させることで作り出すものです。


手法（例：線形回帰、SVM、決定木、ニューラルネットワークなど）
ハイパーパラメータ（例：SVMのカーネルの種類、重みの初期値など）
入力データの前処理の仕方（例：標準化、対数変換、PCAなど）

重要なのはそれぞれのモデルが大きく異なることです。


回帰問題でのブレンディングは非常に単純であるため、scikit-learnには用意されていません。


《補足》


分類問題の場合は、多数決を行います。回帰問題に比べると複雑なため、scikit-learnにはVotingClassifierが用意されています。


sklearn.ensemble.VotingClassifier — scikit-learn 0.21.3 documentation

# 問題1

単一モデルの精度出力

ブレンディングモデルの精度出力

2つ例を示します。

## 単一モデル

In [4]:
# 使用モデル一覧
models = [
    LinearRegression(),
    SVR(),
    DecisionTreeRegressor()
]

# 使用モデルでループ
for model in models:
    # 学習
    model.fit(X_train,y_train)
    # 予測
    prediction = model.predict(X_test)
    # MSE算出
    mse = mean_squared_error(y_test,prediction)
    # print('CV_MSE:',result)
    print('モデル名：%s　MSE：%s'%(str(model),mse))

モデル名：LinearRegression()　MSE：0.062158929372223067
モデル名：SVR()　MSE：0.05880369910471196
モデル名：DecisionTreeRegressor()　MSE：0.08350034833340558


## 複数モデル（例①）

別モデルを3つ学習

In [5]:
# 3つのモデルを個別に学習
model1 = LinearRegression().fit(X_train,y_train)
model2 = SVR().fit(X_train,y_train)
model3 = DecisionTreeRegressor().fit(X_train,y_train)

# 学習したモデルで予測
prediction1 = model1.predict(X_test)
prediction2 = model2.predict(X_test)
prediction3 = model3.predict(X_test)


# 予測値の平均値をとる
prediction = np.mean([prediction1,prediction2,prediction3],axis=0)

# MSEの算出
mse = mean_squared_error(y_test,prediction)

print('モデル名：%s　MSE：%s'%("線形回帰/SVM/決定木",mse))

モデル名：線形回帰/SVM/決定木　MSE：0.056741018991756854


## 複数モデル（例②）

同じモデルのパラメータを変えて学習

In [6]:
# 3つのモデルを個別に学習
model1 = DecisionTreeRegressor(max_depth=3).fit(X_train,y_train)
model2 = DecisionTreeRegressor(max_depth=5).fit(X_train,y_train)
model3 = DecisionTreeRegressor(max_depth=7).fit(X_train,y_train)

# 学習したモデルで予測
prediction1 = model1.predict(X_test)
prediction2 = model2.predict(X_test)
prediction3 = model3.predict(X_test)


# 予測値の平均値をとる
prediction = np.mean([prediction1,prediction2,prediction3],axis=0)

# MSEの算出
mse = mean_squared_error(y_test,prediction)

print('モデル名：%s　MSE：%s'%("決定木",mse))

モデル名：決定木　MSE：0.060553650373665974


## 問題2】バギングのスクラッチ実装
バギング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。


バギングとは
バギングは入力データの選び方を多様化する方法です。訓練データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ ブートストラップサンプル ）を作り出します。それらによってモデルをN個学習し、推定結果の平均をとります。ブレンディングと異なり、それぞれの重み付けを変えることはありません。


sklearn.model_selection.train_test_split — scikit-learn 0.21.3 documentation


推定結果の平均をとる部分はブレンディングと同様の実装になります。

# 問題2

## 単一モデル

In [7]:
model = DecisionTreeRegressor().fit(X_train,y_train)
prediction = model.predict(X_test)
mse = mean_squared_error(y_test,prediction)
print('モデル名：%s　MSE：%s'%("決定木",mse))

モデル名：決定木　MSE：0.08504807149276056


## バギング（例①）

In [8]:
# 何回データセットを作り直すか
n = 20

# 学習したモデルの格納
models = []

for i in range(n):
    # 訓練データをさらにランダムに分割
    X_bagging, _X, y_bagging, _y = train_test_split(X_train,y_train,train_size=0.2,shuffle=True)
    # モデルの定義
    model = DecisionTreeRegressor()
    # 学習
    model.fit(X_bagging,y_bagging)
    # モデルを記録
    models.append(model)

In [9]:
# 予測値の初期値
prediction = np.zeros(len(X_test))

# 記録してあるモデルでループ
for model in models:
    # 予測値の算出
    _prediction = model.predict(X_test)
    # 予測値を加算していく
    prediction += _prediction

# 加算していった予測値をn（モデル数）で割ることで平均値を算出
prediction = prediction/n

# MSE算出
mse = mean_squared_error(y_test,prediction)

print('モデル名：%s　MSE：%s'%("決定木",mse))

モデル名：決定木　MSE：0.06006485092501557


## 【問題3】スタッキングのスクラッチ実装
スタッキング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。


スタッキングとは
スタッキングの手順は以下の通りです。最低限ステージ0とステージ1があればスタッキングは成立するため、それを実装してください。まずは $K_0=3, M_0=2$ 程度にします。


《学習時》


（ステージ $0$ ）


訓練データを $K_0$ 個に分割する。
分割した内の $(K_0 - 1)$ 個をまとめて訓練データ、残り $1$ 個を推定用データとする組み合わせが $K_0$ 個作れる。
あるモデルのインスタンスを $K_0$ 個用意し、異なる訓練データを使い学習する。
それぞれの学習済みモデルに対して、使っていない残り $1$ 個の推定用データを入力し、推定値を得る。（これをブレンドデータと呼ぶ）
さらに、異なるモデルのインスタンスも $K_0$ 個用意し、同様のことを行う。モデルが $M_0$ 個あれば、 $M_0$ 個のブレンドデータが得られる。

（ステージ $n$ ）


ステージ $n-1$ のブレンドデータを$M_{n-1}$ 次元の特徴量を持つ訓練データと考え、 $K_n$ 個に分割する。以下同様である。

（ステージ $N$ ）＊最後のステージ


ステージ $N-1$ の $M_{N-1}$ 個のブレンドデータを$M_{N-1}$ 次元の特徴量の入力として、1種類のモデルの学習を行う。これが最終的な推定を行うモデルとなる。

《推定時》


（ステージ $0$ ）


テストデータを $K_0×M_0$ 個の学習済みモデルに入力し、$K_0×M_0$ 個の推定値を得る。これを $K_0$ の軸で平均値を求め $M_0$ 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ $n$ ）


ステージ $n-1$ で得たブレンドテストを $K_n×M_n$ 個の学習済みモデルに入力し、$K_n×M_n$ 個の推定値を得る。これを $K_n$ の軸で平均値を求め $M_0$ 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ $N$ ）＊最後のステージ


ステージ $N-1$ で得たブレンドテストを学習済みモデルに入力し、推定値を得る。

# 問題3

In [10]:
class Stacking():
    """スタッキング
    """
    def __init__(self, max_depth, splits, models):
        """コンストラクタ
        Parameters
        ----------
        max_depth : スタッキングの最大深さ
        splits : データ分割数
        models : 使用モデル一覧
        fit_models : 学習済みモデル保存用リスト
        """
        self.max_depth = max_depth
        self.n_splits = splits
        self.models = models
        self.fit_models = []
        
    def calc_blending(self,X,y,model):
        """ブレンドデータ生成
        Parameters
        ----------
        X : 説明変数
        y : 目的変数
        m : モデル
        """
        # blendingの初期化
        blending = np.zeros(len(X))
        
        # データの分割
        kf = KFold(n_splits=self.n_splits, shuffle=False)
        
        # 分割されたインデックスでループ
        for train_index, test_index in kf.split(X):
            # データ分割
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            y_train = y_train.ravel()
            y_test = y_test.ravel()
            # 学習
            model.fit(X_train, y_train)
            # 学習済みモデルの保存
            self.fit_models.append(model)
            # テストデータのインデックスに予測値を格納
            blending[test_index] = model.predict(X_test)
            
        return blending
    
    def fit(self,X,y,depth):
        """該当深さの学習実行
        Parameters
        ----------
        X : 説明変数
        y : 目的変数
        depth : スタッキングの「現在の」深さ
        """
        # スタッキングの「現在の」深さをメンバ変数化
        self.depth=depth
        
        # 最大深さまで到達していれば、その深さのモデルを学習させて処理終了
        if self.depth == self.max_depth:
            # 当該深さでのモデルを取得
            self.model = self.models[self.depth]
            # 学習
            self.model.fit(X,y)
            return
        
        # 当該深さでのモデルを取得
        models = self.models[self.depth]
        self.blending = np.zeros([len(X),len(models)])
        
        # この階層の全てのモデルで学習
        for i,model in enumerate(models):
            _blending = self.calc_blending(X,y,model)
            self.blending[:,i] = _blending
        
        # 再帰学習
        # コンストラクタ生成の際の引数は同じ
        self.stk = Stacking(self.max_depth, self.n_splits, self.models)
        
        # 学習実行の際は、ブレンドデータを説明変数として渡す。深さも1加えてやる
        self.stk.fit(self.blending,y,depth+1)
        
    def predict(self,X):
        """予測
        Parameters
        ----------
        X : 説明変数
        y : 目的変数
        depth : スタッキングの「現在の」深さ
        """
        # 最大深さの場合は最終的な予測値を出力するのみ
        if self.depth == self.max_depth:
            # 予測
            prediction = self.model.predict(X)
            # 返す
            return prediction
        # 最大深さに達していない場合は、再帰的に呼び出す
        else:
            
            # 予測値を0で初期化
            self.prediction = np.zeros(len(X))
            # 次の階層に渡すブレンドデータ(仮)の作成
            self.blending = np.zeros([len(X),len(self.models[self.depth])])
            # この階層の学習済みモデルでループ
            count = 0 # 現在どのモデルを回しているか把握
            for model in self.fit_models:
                # 予測し、0で初期化している予測値に加算
                self.prediction += model.predict(X)
                # 1種類のモデル学習の終了判定
                count+=1
                if count%self.n_splits == 0:
                    # 予測値を加算してきたので割って平均値を算出
                    self.prediction = self.prediction/self.n_splits
                    # その平均値を次の階層で使用する説明変数に格納
                    self.blending[:,int(count/self.n_splits)-1] = self.prediction
                    # 次の種類のモデルでの予測のため、予測値は初期化しておく
                    self.prediction = np.zeros(len(X))
            # 次の階層の予測関数
            prediction = self.stk.predict(self.blending)
            return prediction

In [11]:
models = {
    0:[LinearRegression(),DecisionTreeRegressor(),RandomForestRegressor()],
    1:[ARDRegression(),SGDRegressor(),DecisionTreeRegressor()],
    2:[HuberRegressor(),ARDRegression(),RandomForestRegressor()],
    3:LinearRegression()
}

stk = Stacking(max_depth=3,splits=5,models=models)
stk.fit(X_train,y_train,0)

In [12]:
prediction = stk.predict(X_test)

In [13]:
# Rating
mse = mean_squared_error(y_test,prediction)
print('MSE : {:.3f}'.format(mse))

MSE : 0.059
